# Using functional connectivity from the HCP to predict individual characteristics

Connectivity ML Group @ [Neurohackademy 2021](https://neurohackademy.org/)

**Table of Content**
 - Import
 - Load the data
 - Visualization
 - Optimization



# But first, import!

In [ ]:
# our core libraries
import os # (from PL)
import math
import numpy as np
import pandas as pd
import neuropythy as ny
import nibabel as nib
import ipyvolume as ipv
import random
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV # (from TPA and PL)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,plot_confusion_matrix # (from TPA and PL)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder # (from PL)
from sklearn import svm # (from TPA and PL)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import SGDRegressor, LogisticRegression
from sklearn.ensemble import RandomForestClassifier # (from TPA and PL)

import shap  # package used to calculate Shap values
import itertools # (from PL)
import networkx as nx # (from YZ)

# (from DB)

# (from PK)


In [ ]:
# You need to configure neuropythy so that it knows what your
# HCP AWS S3 access key and secret are:
# key = 'YOURKEY'
# secret = 'YOURSECRET'

ny.config['hcp_credentials'] = (key, secret)

ny.config['hcp_auto_download'] = True
ny.config['hcp_auto_path'] = '~/hcp_data'

# Next, load the data..

To `netmaps_df` we load "netmaps" which are subject-specific “parcellated connectomes” – for each subject, a nodes x nodes network matrix. See more [here](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP1200-DenseConnectome+PTN+Appendix-July2017.pdf).

To `behavioral_df` we load the data keys. See more [here](https://wiki.humanconnectome.org/display/PublicData/HCP-YA+Data+Dictionary-+Updated+for+the+1200+Subject+Release). 

In [ ]:
N = 15 # number of ICAs - 15, 25, 50 ,100 ,200 , 300

In [ ]:
netmaps_df = pd.read_csv('data/connectivityml/HCP_PTN1200/netmats/3T_HCP1200_MSMAll_d'+str(N)+'_ts2/netmats2.txt', delim_whitespace=True,header=None)
print("Network-matrices data shape:", netmaps_df.shape)
netmaps_df.head()

In [ ]:
behavioral_df = pd.read_csv('data/connectivityml/unrestricted_pkalra_7_26_2021_17_39_25.csv')
print("Behaviora data shape:", behavioral_df.shape)
behavioral_df.head()

We have netmaps for 1003 subjects so we will need to filter `behavioral_df` a little.

To `subjectsID_df` we load the ordered list of all subjects with complete rfMRI data (recon 1 + recon2) included in this PTN release

In [ ]:
subjectsID_df = pd.read_csv('data/connectivityml/HCP_PTN1200/subjectIDs.txt',header=None,names=["Subject"])
print("Subjects ID data shape:", subjectsID_df.shape)
subjectsID_df.head()

We can see that this corresponds to the number of netmaps we have.

In [ ]:
filter_behavioral_df = subjectsID_df.merge(behavioral_df, on='Subject', how='inner')

print("Filtered behaviora data shape:", filter_behavioral_df.shape)
filter_behavioral_df.head()

## Pre-process features matrix

In [ ]:
netmapsX_df = pd.DataFrame(data = netmaps_df, columns = range(N*N))
netmapsX_df = netmapsX_df.T.drop_duplicates(keep='first').T
netmapsX_df = netmapsX_df.T.drop_duplicates(keep='last').T
X = netmapsX_df
print("Features matrix shape:", X.shape)
X.head()

## Pre-process predicted values

Here we are going to foucs on the subject gender.

In [ ]:
filter_behavioral_df['Gender_i']=np.zeros(shape=(subjectsID_df.shape))
filter_behavioral_df.Gender_i = pd.factorize(filter_behavioral_df.Gender)[0] # Encode the object as an enumerated type or categorical variable.
y_gender = filter_behavioral_df.Gender_i # Gender of Subject
print("y_gender shape:", y_gender.shape)

filter_behavioral_df['Gender_i'].groupby(filter_behavioral_df['Gender']).unique().apply(pd.Series).rename(columns={0:'Labels'}).sort_values(by='Labels')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(y_gender, ax=ax)
ax.set_title("Gender of Subject")
fig.tight_layout()

## Time for Random forests!

Data and estimators exploration was done in a separate notebook..

### [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

Here we will prdict the gender of subjects in our dataset from our network-matrices data has gave us the best results...

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y_gender, test_size=0.2,stratify=y_gender,random_state=1)

In [ ]:
rf = RandomForestClassifier(random_state=1)

rf.fit(xtrain, ytrain) # Build a forest of trees from the training set (X, y).

score = rf.score(xtest, ytest) # Return the mean accuracy on the given test data and labels.
print("Test set score: ", score) 

ypred = rf.predict(xtest) # Predict class for X.

In [ ]:
# View confusion matrix for test data and predictions
plot_confusion_matrix(rf, xtest, ytest) # Plot Confusion Matrix.
plt.savefig('output/confusion_matrix_'+str(N)+'.png')

In [ ]:
cr = classification_report(ytest, ypred,output_dict=True) # Build a text report showing the main classification metrics.
sns_plot = sns.heatmap(pd.DataFrame(cr).iloc[:-1, :].T, annot=True) # plot scikit-learn classification report
sns_plot.figure.savefig("output/classification_report_"+str(N)+".png")

#### Visualizing trees
We can plot individual decision trees - 

In [ ]:
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image, display

def plot_graphviz_tree(tree):
    """
    Helper function that takes a tree as input, calls sklearn's export_graphviz
    function to generate an image of the tree using graphviz, and then
    plots the result in-line.
    """
    export_graphviz(tree, out_file='tree.dot', max_depth=3, filled=True,
                    feature_names=X.columns, impurity=False, rounded=True,
                    proportion=False, precision=2);

    call(['dot', '-Tpng', 'tree.dot', '-o', 'output/tree_'+str(N)+'.png', '-Gdpi=600'])
    display(Image(filename = 'output/tree_'+str(N)+'.png'));

In [ ]:
# First tree in the forest
plot_graphviz_tree(rf.estimators_[0]);

#### Interpreting random forests

##### Feature importances

Unlike regression-based methods, random forests don't have linear coefficientsso we look at the feature importances, to tell us how each feature contributes to the overall prediction.

In [ ]:
f = open('output/feature_importances_'+'_'+str(N)+'.txt', "w")
f.write(str(pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False).head(10)))
f.close()

# plot feature importances
(pd.Series(rf.feature_importances_, index=X.columns)
   .nlargest(10)
   .plot(kind='bar',title="Feature importances for "+str(N)+ " ICA's"))
plt.savefig('output/feature_importances_'+str(N)+'.png')

##### SHAP values
To interpret our results we will look at the SHAP values of our model.
See more [here](https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a).

SHAP feature importance is an alternative to standard feature importance based on magnitude of feature attributions. The feature importance is useful, but contains no information beyond the importances. For a more informative plot, we will next look at the summary plot.

The SHAP summary plot is made of all the dots in the test data. Showing:
- **Feature importance:** Variables are ranked in descending order.
- **Impact:** The horizontal location shows whether the effect of that value is associated with a higher or lower prediction.
- **Original value:** Color shows whether that variable is high (in red) or low (in blue) for that observation.


In [ ]:
# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of xtest rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(xtest)

# Make plot
fig = shap.summary_plot(shap_values[1], xtest, show=False)
plt.savefig('output/summary_plot_xtest_'+str(N)+'.png')

In [ ]:
fig = shap.summary_plot(shap_values, xtest, show=False,plot_type='bar')
plt.savefig('output/SHAP_feature_importances_xtest_'+str(N)+'.png')

#### Single subject interpretation
We will look at SHAP values for a random row of the dataset.

##### SHAP values
For context, we'll look at the raw predictions before looking at the SHAP values.

In [ ]:
row_to_show = 5
data_for_prediction = xtest.loc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

rf.predict_proba(data_for_prediction_array)

In [ ]:
print("Subject evaluated:",subjectsID_df.Subject[row_to_show])

In [ ]:
# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)
shap.initjs()

shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction, show=False)
# plt.savefig('force_plot_hcp_'+str(subjectsID_df.Subject[row_to_show])+'_'+str(N)+'.png')

The SHAP values of all features sum up to explain why the prediction was different from the baseline. This allows us to decompose a prediction in a graph like this where:

- The **output value** is the prediction for that observation (the prediction of the subject evluated).
 - The **base value** is the value that would be predicted if we did not know any features for the current output (the mean prediction, or mean(yhat)).
 - **Red/blue:** Features that push the prediction higher (to the right) are shown in red, and those pushing the prediction lower are in blue.
 
##### TOP 5 Most important ICA's

In [ ]:
vals= np.abs(shap_values).mean(0)
feature_importance = pd.DataFrame(list(zip(xtest.columns,vals)),columns=['netmat_col','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True)
feature_importance.head()

In [ ]:
edge_df = pd.DataFrame(columns = ['netmat_col','Node1', 'Node2', 'Weight'])
countlist = list() # if node1,node2 weight is saved no need to save node2,node1

for index, row in tqdm(netmaps_df.iterrows(), total=netmaps_df.shape[0]):
    netmat_col = 0
    if index == row_to_show:
        row2mat = row.values.reshape(N,N)
        for node1 in range(N):
            for node2 in range(N):
                if node1!=node2:
                    if (node2, node1) not in countlist:
                        countlist.append((node1, node2))
                        curr_edge = {'netmat_col': netmat_col, 'Node1': node1, 'Node2': node2, 'Weight':row2mat[node1][node2]}
                        edge_df = edge_df.append(curr_edge, ignore_index = True)
                netmat_col = netmat_col + 1

In [ ]:
f = open('output/top5_'+str(subjectsID_df.Subject[row_to_show])+'_'+str(N)+'.txt', "w")

for i in range(5):
    feature = feature_importance.netmat_col.values[i]
    print("Important feature #: ",feature)
    f.write("Important feature #: " + str(feature)+"\n")
    ICA_Node1 = edge_df.loc[edge_df['netmat_col'] == feature, 'Node1']
    ICA_Node2 = edge_df.loc[edge_df['netmat_col'] == feature, 'Node2']

    print("Node 1:",ICA_Node1.iloc[0])
    f.write("Node 1: " + str(ICA_Node1.iloc[0])+"\n")
    print("Node 2:",ICA_Node2.iloc[0])
    f.write("Node 2: " + str(ICA_Node2.iloc[0])+"\n")
f.close()

Knowing the ICA's that are part of the most important feature we can plot them on our single subject brain

In [ ]:
# Get a sample HCP subject:
sub = ny.hcp_subject(subjectsID_df.Subject[row_to_show])
sub

In [ ]:
# Load the CIFTI file:
cii_filename = '~/data/connectivityml/HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d'+str(N)+'.ica/melodic_IC.dscalar.nii'
cii_obj = ny.load(cii_filename)

# Split the CIFTI object into hemisphere/subvoxel data:
(lh_data, rh_data, subvox_data) = ny.hcp.cifti_split(cii_obj)

# These data should be (N(data-points) x vertices)
lh_data.shape

In [ ]:
# sub.lh and sub.rh are the "native" (FreeSurfer) hemispheres;
# sub.hemis['lh_LR32k'] and sub.hemis['rh_LR32k'] are the
# HCP subject-aligned fs_LR hemispheres (with 32k resolution).
lh_hemi_native = sub.lh
rh_hemi_native = sub.rh

# The 32492 size indicates this is a 32k LR hemisphere:
lh_hemi = sub.hemis['lh_LR32k']
rh_hemi = sub.hemis['rh_LR32k']

**ICA 1**

In [ ]:
# We can make an ipyvolume figure to plot both hemispheres on:
fig = ipv.figure()
# Then plot each hemisphere using whichever ICA component we
# want to visualize:

ICA_Node1 = int(edge_df.loc[edge_df['netmat_col'] == feature_importance.netmat_col.values[0], 'Node1'])
ny.cortex_plot(lh_hemi, surface='inflated', color=lh_data[ICA_Node1], cmap='hot', figure=fig)
ny.cortex_plot(rh_hemi, surface='inflated', color=rh_data[ICA_Node1], cmap='hot', figure=fig)

In [ ]:
ipv.pylab.save('output/most_important_ICA1_'+str(subjectsID_df.Subject[row_to_show])+'_'+str(N)+'.html',title='Most important feature ICA 1')

**ICA 2**

In [ ]:
# We can make an ipyvolume figure to plot both hemispheres on:
fig = ipv.figure()
# Then plot each hemisphere using whichever ICA component we
# want to visualize:

ICA_Node2 = int(edge_df.loc[edge_df['netmat_col'] == feature_importance.netmat_col.values[0], 'Node2'])
ny.cortex_plot(lh_hemi, surface='inflated', color=lh_data[ICA_Node2], cmap='hot', figure=fig)
ny.cortex_plot(rh_hemi, surface='inflated', color=rh_data[ICA_Node2], cmap='hot', figure=fig)

In [ ]:
ipv.pylab.save('output/most_important_ICA2_'+str(subjectsID_df.Subject[row_to_show])+'_'+str(N)+'.html',title='Most important feature ICA 2')

# Visualization

## Brain-free network model using Gephi

See more about Gephi [here](https://gephi.org/). The following primarily follows [this instruction](https://towardsdatascience.com/from-csv-to-github-pages-in-5-steps-publishing-an-interactive-social-network-of-the-marvel-7b8374bf44fb). There is a tutorial about using the `GephiStreamer` python package [here](https://pypi.org/project/GephiStreamer/), but that has to be run with Graph streaming plugin enabled, although there is an [instruction](https://tbgraph.wordpress.com/2017/04/01/neo4j-to-gephi/) it's not straightforward to set this up in NeuroHackademy Jupyterhub.

In [ ]:
edge_df['weight'] = 1

for ind in (feature_importance.netmat_col[:5].index):
    print(ind)
    edge_df.loc[ind-1,'weight'] = 5
edge_df

Make sure that the weights from the top features are changed correctly for visualization:

In [ ]:
edge_df.loc[edge_df['netmat_col'].isin(feature_importance.netmat_col[:5])]

In [ ]:
G=nx.from_pandas_edgelist(edge_df, 'Node1', 'Node2', edge_attr='weight')

In [ ]:
# number of ICAs - 15, 25, 50 ,100 ,200 , 300

nx.write_gexf(G, "/home/jovyan/Hackathon/code/tpatpa/connectivityml/gephi-15-top5-weighted.gexf")

![Top 5 features from 15 ICAs, labelled](visualization-gephi/gephi-15-top5-weighted-labelled.png)

## Brain Mapping using Neuropythy

In [ ]:
# To be added

# Optimization

In [ ]:
# Load the connecitvity data as a dataframe
DataDir= '/home/jovyan/data/connectivityml/HCP_PTN1200/netmats'

# Load connecitivty data as dictionaries
Folders_list = os.listdir(DataDir)
Conn_data1 = dict();
Conn_data2 = dict();
j=0; Nodes = list();
for flds in Folders_list:
    Nodes.append(flds.split("_")[3])
    net1 = np.loadtxt(os.path.join(DataDir,flds,'netmats1.txt'))
    net2 = np.loadtxt(os.path.join(DataDir,flds,'netmats2.txt'))
    N  = int(Nodes[j][1:])
    indx = np.triu_indices(N,1)
    mat1 = np.transpose(np.transpose(net1.reshape(1003,N,N),[1,2,0])[indx],[1,0])
    mat2 = np.transpose(np.transpose(net2.reshape(1003,N,N),[1,2,0])[indx],[1,0])
    Conn_data1[Nodes[j]] = mat1
    Conn_data2[Nodes[j]] = mat2
    j=j+1



# Loading the behavioral data
Behv_data_df = pd.read_csv('/home/jovyan/ML-Proj/unrestricted_pkalra_7_26_2021_17_39_25.csv')

# Loading Subject IDs
SubjectID= np.loadtxt("/home/jovyan/ML-Proj/subjectIDs.txt")

In [ ]:
#Loading and selecting subjects in behavioral data based on available RS data
Behv_data_df = pd.read_csv('/home/jovyan/ML-Proj/unrestricted_pkalra_7_26_2021_17_39_25.csv')
Behv_data_df = Behv_data_df.set_index('Subject')
Behv_data_df = Behv_data_df.loc[SubjectID,:]

In [ ]:
# Choosing a predictor (We chose gender)
y= Behv_data_df["Gender"]
y= y.to_numpy()

In [ ]:
# Optimization using nested CV

hyp1 = dict();
hyp2 = dict();
hyp1['C'] =[0.1,1,10] # Hyperparameters for SVM
hyp2['n_estimators'] = [1,10,100] # Hyperparameters for SVM

SVM_results_IC1= dict();
RF_results_IC1= dict();
SVM_results_IC2= dict();
RF_results_IC2= dict();

for j in np.arange(len(Folders_list)):
    X1= Conn_data1[Nodes[j]]
    X2= Conn_data2[Nodes[j]]
    clf1 = svm.SVC(kernel='linear')
    clf2 = RandomForestClassifier(max_depth =2)
    outer_cv = KFold(n_splits=3, shuffle=True)
    SVM_results1= list()
    RF_results1= list()
    SVM_results2= list()
    RF_results2= list()
    for train_ix, test_ix in outer_cv.split(X1):
        # split data into train, test for outer CV
        X1_train, X1_test = X1[train_ix], X1[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        X2_train, X2_test = X2[train_ix], X2[test_ix]
        inner_cv=  KFold(n_splits=3, shuffle=True)       
        Search1 = GridSearchCV(estimator=clf1,scoring='accuracy', param_grid=hyp1, cv=inner_cv,refit=True)
        Search2 = GridSearchCV(estimator=clf2,scoring='accuracy', param_grid=hyp2, cv=inner_cv,refit=True)
        # execute search
        resultSVM1 =  Search1.fit(X1_train, y_train)
        resultRF1 =  Search2.fit(X1_train, y_train)
        resultSVM2 =  Search1.fit(X2_train, y_train)
        resultRF2 =  Search2.fit(X2_train, y_train)
        # get the best performing model fit on the whole training set
        best_SVMmodel1 = resultSVM1.best_estimator_
        best_SVMmodel2 = resultSVM2.best_estimator_
        best_RFmodel1 = resultRF1.best_estimator_
        best_RFmodel2 = resultRF2.best_estimator_
        # evaluate the best model on the hold out dataset
        yhatSVM1 = best_SVMmodel1.predict(X1_test)
        yhatRF1 = best_RFmodel1.predict(X1_test)
        yhatSVM2= best_SVMmodel2.predict(X2_test)
        yhatRF2= best_RFmodel2.predict(X2_test)
        # Calculate the accuracy
        SVMacc1= accuracy_score(y_test,yhatSVM1)
        RFacc1= accuracy_score(y_test, yhatRF1)
        SVMacc2= accuracy_score(y_test,yhatSVM2)
        RFacc2= accuracy_score(y_test, yhatRF2)
        # store the result for each method
        SVM_results1.append(SVMacc1)
        RF_results1.append(RFacc1)
        SVM_results2.append(SVMacc2)
        RF_results2.append(RFacc2)
        
    # store the results for all IC parcellations
    SVM_results_IC1[Nodes[j]]= SVM_results1
    RF_results_IC1[Nodes[j]]= RF_results1
    SVM_results_IC2[Nodes[j]]= SVM_results2
    RF_results_IC2[Nodes[j]]= RF_results2

In [ ]:
# Tabulating the results
SVMacc1 =  pd.DataFrame(SVM_results_IC1)
RFacc1 =  pd.DataFrame(RF_results_IC1)
SVMacc2 =  pd.DataFrame(SVM_results_IC2)
RFacc2=  pd.DataFrame(RF_results_IC2)
SVMacc1.index = hyp1['C']
SVMacc2.index = hyp1['C']
RFacc1.index = hyp2['n_estimators']
RFacc2.index = hyp2['n_estimators']
print("\n Correlation\n")
print("SVM\n")
print(SVMacc1)
print("\nRF\n")
print(RFacc1)
print("\n Partial correlation\n")
print("SVM\n")
print(SVMacc2)
print("\nRF\n")
print(RFacc2)

In [ ]:
# Plotting the results using 3D barplots
fig = plt.figure(figsize=(10,10),dpi=80)
ax1= fig.add_subplot(2,2,1, projection='3d')
ax2= fig.add_subplot(2,2,2, projection='3d')
ax3= fig.add_subplot(2,2,3, projection='3d')
ax4= fig.add_subplot(2,2,4, projection='3d')

lx1= len(hyp1['C'] )
ly = len(Nodes)
lx2= len(hyp2['n_estimators'])

xpos1= np.arange(0, lx1,1)
xpos2= np.arange(0, lx2,1)
ypos= np.arange(0, ly, 1)
xpos1g,yposg=  np.meshgrid(xpos1,ypos)
xpos2g,yposg=  np.meshgrid(xpos2,ypos)


# Setting x, y and z positions
xpos1g = xpos1g.flatten()
yposg= yposg.flatten()
xpos2g = xpos2g.flatten()
zpos1g=np.zeros(lx1*ly)
zpos2g=np.zeros(lx2*ly)

# Making the depths 
dx1= 0.5*np.ones_like(zpos1g)
dx2= 0.5*np.ones_like(zpos2g)
dy1= 0.5*np.ones_like(zpos1g)
dy2= 0.5*np.ones_like(zpos2g)

dz11=SVMacc1.values.flatten()
dz12=SVMacc2.values.flatten()
dz21=RFacc1.values.flatten()
dz22=RFacc2.values.flatten()


ax1.bar3d(xpos1g,yposg,zpos1g,dx1,dy1,dz11)
ax2.bar3d(xpos1g,yposg,zpos1g,dx1,dy1,dz12)
ax3.bar3d(xpos2g,yposg,zpos2g,dx2,dy2,dz21)
ax4.bar3d(xpos2g,yposg,zpos2g,dx2,dy2,dz22)

ax1.w_xaxis.set_ticks(np.arange(lx1))
ax1.w_xaxis.set_ticklabels(hyp1['C'])
ax1.w_yaxis.set_ticks(np.arange(ly))
ax1.w_yaxis.set_ticklabels(Nodes)

ax2.w_xaxis.set_ticks(np.arange(lx1))
ax2.w_xaxis.set_ticklabels(hyp1['C'])
ax2.w_yaxis.set_ticks(np.arange(ly))
ax2.w_yaxis.set_ticklabels(Nodes)

ax3.w_xaxis.set_ticks(np.arange(lx2))
ax3.w_xaxis.set_ticklabels(hyp2['n_estimators'])
ax3.w_yaxis.set_ticks(np.arange(ly))
ax3.w_yaxis.set_ticklabels(Nodes)

ax4.w_xaxis.set_ticks(np.arange(lx2))
ax4.w_xaxis.set_ticklabels(hyp2['n_estimators'])
ax4.w_yaxis.set_ticks(np.arange(ly))
ax4.w_yaxis.set_ticklabels(Nodes)


ax1.set_xlabel('C')
ax1.set_ylabel('IC type')
ax1.set_zlabel('Accuracy')
ax1.title.set_text('SVM-Corr')

ax2.set_xlabel('C')
ax2.set_ylabel('IC type')
ax2.set_zlabel('Accuracy')
ax2.title.set_text('SVM-Pcorr')

ax3.set_xlabel('No of trees')
ax3.set_ylabel('IC type')
ax3.set_zlabel('Accuracy')
ax3.title.set_text('RF-Corr')

ax4.set_xlabel('No of trees')
ax4.set_ylabel('IC type')
ax4.set_zlabel('Accuracy')
ax4.title.set_text('RF-Pcorr')


plt.show()